# Foocus Installation

In [ ]:
HOME_FOLDER = '/kaggle/working'
TEMP_FOLDER = '/kaggle/temp'
update = False

checkpoints = f'{HOME_FOLDER}/Fooocus/models/checkpoints'
temp_models = f'{HOME_FOLDER}/Fooocus/models/checkpoints/temp_models'

!mkdir $TEMP_FOLDER

import os
from os import path

%cd $HOME_FOLDER

if not path.exists('Fooocus'):
    get_ipython().system('git clone https://github.com/Afylx01/Fooocus.git')
else:
    get_ipython().system(f'find {HOME_FOLDER}/Fooocus/models/checkpoints -maxdepth 1 -type l -delete') # delete any symlinks to temp models from a previous run
    
%cd Fooocus

if update:
    get_ipython().system('git pull')
    
!pip install -r requirements_versions.txt 
!pip install torch torchvision --force-reinstall --index-url https://download.pytorch.org/whl/cu117
!rm -rf /opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/PyYAML-6.0.1.dist-info
!pip install pyyaml pytz 

!mamba install openssh -y

# Install Models

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://huggingface.co/afylx/mohawk/resolve/main/MOHAWK_v20BackedVAE.safetensors?download=true'
model_name = 'MOHAWK_v20BackedVAE.safetensors'

CHECK_FOLDER = HOME_FOLDER + '/Fooocus/models/checkpoints'
%cd $CHECK_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Install a LoRA

In [ ]:
# Install a LoRA to permanent storage
model_url = 'https://civitai.com/api/download/models/171989?type=Model&format=SafeTensor'
model_name = 'Detail-Slider-LoRA.safetensors'

LORA_FOLDER = HOME_FOLDER + '/Fooocus/models/loras'
%cd $LORA_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Web UI with ngrok

In [ ]:
# Starting the Web UI with ngrok

# --- Variables ---

Ngrok_token = "2did5JQqpE2CkZnzxlwnlywyHYA_67ed5dYoq951q7UxWJcn4" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)
# Example: Ngrok_token = "2Fw13n4GcJT12g7mSDUC62cdNGb_5svdjf3Gg5vfhr4nGr5gF"

Ngrok_domain = "" # optional, leave empty if you don't have a domain

port = 7865

# -----------------


!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys
import gc

gc.collect()

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(port , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system(f"python {HOME_FOLDER}/Fooocus/entry_with_update.py ")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

# Web UI with RemoteMoe

In [ ]:
#Option 2: Starting the Web UI with RemoteMoe
port = 7865

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python $HOME_FOLDER/Fooocus/entry_with_update.py --preset lightning01 & ssh -R 80:127.0.0.1:$port -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

100%|███████████████████████████████████████████| 30/30 [00:25<00:00,  1.18it/s]
Requested to load AutoencoderKL
Loading 1 new model
[Fooocus Model Management] Moving model(s) has taken 1.47 seconds


In [ ]:
!pip install --upgrade numpy

# Zip output

In [ ]:
# @title 7. Make zip of the outputs in here
import shutil

# Specify the path to the directory you want to zip
directory_path = '/kaggle/working/Fooocus/outputs'

# Specify the path and name of the zip file to create
zip_file_path = '/kaggle/working/outputs/outputs'

# Create a zip file
shutil.make_archive(zip_file_path, 'zip', directory_path)

# Create output folder

In [ ]:
import os

directory_name1 = "outputs"
directory_path1 = "/kaggle/working/" + directory_name1
os.makedirs(directory_path1, exist_ok=True)

directory_name2 = "outputs"
directory_path2 = "/kaggle/working/Fooocus/" + directory_name2
os.makedirs(directory_path2, exist_ok=True)

# Delete outputs

In [ ]:
!rm -rf $HOME_FOLDER/Fooocus/outputs/*
!rm -rf $HOME_FOLDER/outputs/*

# Move files

In [ ]:
import shutil

src_path = r"/kaggle/input/wildcard/boy.txt"
dst_path = r"/kaggle/working/Fooocus/wildcards/"

shutil.move(src_path, dst_path)